Taking Inpuut

In [ ]:
pdf_path=input("enter pdf path:")
from langchain_community.document_loaders import pdf_loader
doc=pdf_loader(pdf_path)
doc

ModuleNotFoundError: No module named 'langchain_community.document_loader'